In [17]:
import nltk
import sys
import math
from decimal import *
import numpy as np

In [18]:
## QUESTION 1

In [33]:
# Bring in data
def init_data(count=10000):
    training_data = nltk.corpus.brown.tagged_sents(tagset='universal')[:count]
    return training_data

In [35]:
#generating all the datasets
def gen_data_sets(training_data):
    tag_set = set()
    word_set = set()
    transition_dict = {}
    tag_count = {}

    for value in training_data:
        previous = "start"
        for data in value:
            word_set.add(data[0].lower())
            tag = data[1]
            tag_set.add(tag)

            if tag in tag_count:
                tag_count[tag] += 1
            else:
                tag_count[tag] = 1

            if (previous, tag) in transition_dict:
                transition_dict[(previous, tag)] += 1
                previous = tag
            else:
                transition_dict[(previous, tag)] = 1
                previous = tag

    return tag_set, word_set, transition_dict, tag_count

In [36]:
# Mapping to indices
def build_word_map(word_set):
    word_map = {'start': 0}
    i = 1
    for word in word_set:
        if word in word_map:
            continue
        else:
            word_map[word] = i
            i += 1
    word_map['undefined'] = len(word_map)

    return word_map

def build_tag_map(tag_set):
    i = 1
    tag_map = {'start': 0}
    for tag in tag_set:
        if tag in tag_map:
            continue
        else:
            tag_map[tag] = i
            i += 1

    return tag_map

In [22]:
# Building transition dictionary
def gen_prob_dict(transition_dict, tag_set, tag_count, word_set):
    prob_dict = {}
    for key in transition_dict:
        den = 0
        val = key[0]
        for key_2 in transition_dict:
            if key_2[0] == val:
                den += transition_dict[key_2]
        prob_dict[key] = Decimal(transition_dict[key])/(den)
    for tag in tag_set:
        if ("start", tag) not in prob_dict:
            prob_dict[("start", tag)] = Decimal(
                1) / Decimal(len(word_set) + tag_count[tag])
    for tag1 in tag_set:
        for tag2 in tag_set:
            if (tag1, tag2) not in prob_dict:
                prob_dict[(tag1, tag2)] = Decimal(
                    1) / Decimal(len(word_set) + tag_count[tag1])
    return prob_dict

In [23]:
# Generating counts of words
def gen_counts(training_data):
    count_word = {}
    for value in training_data:
        for data in value:
            word = data[0]
            tag = data[1]
            if (word, tag) in count_word:
                count_word[(word, tag)] += 1
            else:
                count_word[(word, tag)] = 1
    return count_word

In [24]:
# generating the emission probabilities 
def gen_emission_prob(count_word, tag_count):
    emission_prob_dict = {}
    for key in count_word:
        emission_prob_dict[key] = Decimal(count_word[key])/tag_count[key[1]]

    return emission_prob_dict

In [25]:
# generating matrices
def gen_transition_matrix(tag_map, prob_dict):
    transition_matrix = np.zeros((len(tag_map), len(tag_map)))
    
    for key, value in prob_dict.items():
        i = tag_map[key[0]]
        j = tag_map[key[1]]
        transition_matrix[i, j] = value
    return transition_matrix

def gen_observation_matrix(tag_map, word_map, emission_prob_dict):
    observation_matrix = np.zeros((len(tag_map), len(word_map)+1))

    for key, value in emission_prob_dict.items():
        i = tag_map[key[1]]
        j = word_map[key[0].lower()]

        observation_matrix[i][j] = value

    random_model = 1/len(tag_map)
    for tag in tag_map.keys():
        i = tag_map[tag]
        j = word_map['undefined']

        observation_matrix[i][j] = random_model

    return observation_matrix

In [37]:
# Take in data and return the transition matrix, observation matrix, and the mappings of words to indices
def load_matricies(training_data):
    
    tag_set, word_set, transition_dict, tag_count = gen_data_sets(training_data)
    word_map = build_word_map(word_set)

    tag_map = build_tag_map(tag_set)

    prob_dict = gen_prob_dict(transition_dict, tag_set, tag_count, word_set)

    word_count = gen_counts(training_data)

    emission_prob_dict = gen_emission_prob(word_count, tag_count)


    transition_matrix = gen_transition_matrix(tag_map, prob_dict)
        
    observation_matrix = gen_observation_matrix(tag_map, word_map, emission_prob_dict)

    pi = transition_matrix[0]
    
    a = transition_matrix

    b = observation_matrix

    return pi, a, b, word_map, tag_map

In [38]:
## QUESTION 2

In [44]:
#Building path with viterbi
def viterbi(obs, pi, a, b):
    
    num_states = np.shape(b)[0]
    T = np.shape(obs)[0]

    states = np.zeros(T, dtype=int)
    delta = np.zeros((num_states, T))
    phi = np.zeros((num_states, T))

    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0
    for t in range(1, T):
        for s in range(num_states):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]]
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])

    states[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        states[t] = phi[states[t+1], [t+1]]

    return states

In [45]:
# Helper methods to translate sentences to and from indices
def sentence_to_idx(sentence, word_map):
    idx_sentence = [0]
    for wordtag in sentence:
        word = wordtag[0]
        try:
            idx = word_map[word]
        except:
            idx = word_map['undefined']
        idx_sentence.append(idx)
    return idx_sentence


def idx_to_sentence(states, tag_map):
    final = []
    for state in states:
        for tag in tag_map.keys():
            if tag_map[tag] == state:
                final.append(tag)
    return final[1:]

In [41]:
## QUESTION 3

In [48]:
def test_run():
    train_data = init_data()
    pi, a, b, word_map, tag_map = load_matricies(train_data)
    test = nltk.corpus.brown.tagged_sents(tagset='universal')[10150:10153]
    for t in test:
        print(t)
        indices = sentence_to_idx(t,word_map)
        states = viterbi(indices, pi, a, b)
        print(idx_to_sentence(states, tag_map))
        print()

In [49]:
test_run()

[('Those', 'DET'), ('coming', 'VERB'), ('from', 'ADP'), ('other', 'ADJ'), ('denominations', 'NOUN'), ('will', 'VERB'), ('welcome', 'VERB'), ('the', 'DET'), ('opportunity', 'NOUN'), ('to', 'PRT'), ('become', 'VERB'), ('informed', 'VERB'), ('.', '.')]
['.', 'VERB', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'DET', 'NOUN', 'PRT', 'VERB', 'VERB', '.']

[('The', 'DET'), ('preparatory', 'ADJ'), ('class', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('introductory', 'ADJ'), ('face-to-face', 'ADJ'), ('group', 'NOUN'), ('in', 'ADP'), ('which', 'DET'), ('new', 'ADJ'), ('members', 'NOUN'), ('become', 'VERB'), ('acquainted', 'VERB'), ('with', 'ADP'), ('one', 'NUM'), ('another', 'DET'), ('.', '.')]
['ADP', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'ADP', 'NUM', 'DET', '.']

[('It', 'PRON'), ('provides', 'VERB'), ('a', 'DET'), ('natural', 'ADJ'), ('transition', 'NOUN'), ('into', 'ADP'), ('the', 'DET'), ('life', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('l